# Manhwa

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

title = []
chapters = []
status = []
published = []
genres = []
favorites = []
members = []
popularity = []
rank = []
score = []
authors = []
volumns = []
synopsis = []
n=0

for limit in range(2900, 2950, 50):
    url = f"https://myanimelist.net/topmanga.php?type=manhwa&limit={limit}"
    page = requests.get(url)
    doc = BeautifulSoup(page.content, 'html.parser')

    
    for item in doc.find_all(class_='ranking-list'):
        n+=1
        print(n, end=' ') 
        
        title.append(item.find(class_='hoverinfo_trigger fs14 fw-b').text)
        
        link = item.find(class_='hoverinfo_trigger fs14 fw-b').get('href')
        item_page = requests.get(link)
        item_doc = BeautifulSoup(item_page.content, 'html.parser')
        
        time.sleep(2)
        try:
            synopsis.append(item_doc.find(itemprop="description").text)
        except:
            synopsis.append(None)

        for info in item_doc.find(class_='borderClass').find_all(class_='spaceit_pad'):
            cur_info = info.text.strip().split(':')
            if cur_info[0] == 'Volumns':
                volumns.append(cur_info[1].strip())
            elif cur_info[0] == 'Chapters':
                chapters.append(cur_info[1].strip())
            elif cur_info[0] == 'Status':
                status.append(cur_info[1].strip())
            elif cur_info[0] == 'Published':
                published.append(cur_info[1].strip())
            elif cur_info[0] == 'Genres' or cur_info[0] == 'Genre':
                cur_genre = ''
                genre_list = cur_info[1].split(' ')
                for i in genre_list:
                    if i != '':
                        if i[-1] == ',':
                            cur_genre += i
                cur_genre += genre_list[-1]
                if cur_genre == '':
                    genres.append('Unknown')
                else:
                    genres.append(cur_genre)
            elif cur_info[0] == 'Authors':
                authors.append(cur_info[1].strip())
            elif cur_info[0] == 'Score':
                score.append(cur_info[1].strip().split(' ')[0][:-1])
            elif cur_info[0] == 'Ranked':
                rank.append(cur_info[1].strip().split(' ')[0][:-1])
            elif cur_info[0] == 'Popularity':
                popularity.append(cur_info[1].strip())
            elif cur_info[0] == 'Members':
                members.append(cur_info[1].strip())
            elif cur_info[0] == 'Favorites':
                favorites.append(cur_info[1].strip())
            elif cur_info[0] == 'Volumes':
                volumns.append(cur_info[1].strip())
                
        if len(genres) != len(volumns):
            genres.append('Unknown')

In [ ]:
mal_manhwa_df = pd.DataFrame({'type': 'manhwa', 'title': title, 'chapters': chapters, 'status': status, 
                              'genres': genres, 'favorites': favorites, 'popularity': popularity, 
                              'rank': rank, 'score': score, 'members': members, 'synopsis': synopsis, 
                              'volumns': volumns, 'authors': authors, 'publish_time': published}, 
                              index=list(range(2901,2901+len(score))))

mal_manhwa_df.to_csv('2900-3000manhwa_mal.csv')

In [ ]:
_0_100 = pd.read_csv('0-100manhwa_mal.csv')
file_list = []
df_list = []
for i in range(0,3000,100):
    file = f"{i}-{i+100}manhwa_mal.csv"
    file_list.append(file)
print(file_list)
for i in file_list:
    df_list.append(pd.read_csv(i))
manhwa_df = pd.concat(df_list).drop(['Unnamed: 0'], axis=1)
manhwa_df = manhwa_df.reset_index()
manhwa_df.drop('index',axis=1).to_csv('manhwa_mal.csv')